In [ ]:
%matplotlib inline
from IPython.display import clear_output
from pytesseract import image_to_string

from srbot import *

In [ ]:
#Tree copper and shaft fletcher

logs = load_image('oak_logs.png')
knife = load_image('knife.png')

target()
while True:
    client = get_client()
    if len(find_bitmap(loginscreen,client)) > 0:
        logins = logins + 1
        if logins > 10:
            raise RuntimeError('Too many logins! Bailing out before we get fishy!')
        login()
        continue
            
    if np.random.random() < 0.1:
        set_compass_angle(np.random.random()*360,tol=15)
        
    mainscreen = get_mainscreen()
    
    if count_inv() > 26: #fletch (or fire)
        print('time to fletch')
        inventory = get_inventory()
        k = find_best_bitmap(knife, inventory, mode='dist', tol=0.02)
        l = find_best_bitmap(logs, inventory, mode='dist', tol=0.2)
        print(len(k),len(l))
        if len(k) > 0 and len(l) > 0:
            np.random.shuffle(k)
            np.random.shuffle(l)
            click_mouse(*(k[0]+[ivxs,ivys]))
            sleep(1.0)
            click_mouse(*(l[0]+[ivxs,ivys]))
            sleep(1.0)
            click_mouse(76,405,left=False)
            sleep(1.0)
            click_mouse(55,477)
            sleep(1.0)
            send_keys('9999')
            sleep(1.0)
            send_keys('\n')
            i = 0
            last_len = len(l)
            while True:
                inventory = get_inventory()
                new_len = len(find_best_bitmap(logs, inventory,mode='dist', tol=0.2))
                if new_len == 0:
                    break
                if last_len != new_len:
                    last_len = new_len
                    i = 0
                i=i+1
                if i > 4:
                    print('timed out')
                    break
                sleep(0.5)
        else:
            print('missing knife or logs')
        
    a = find_colors([0,10,0],mainscreen,tol=0.05,mode='hsl')
    c = find_colors([150,170,100],mainscreen,tol=0.05,mode='hsl')
    trees = filter_near(a,c,20)
    clusters,counts = cluster(trees,radius=20)
    trees = np.asarray([np.mean(cluster.T,axis=1) for cluster in clusters])
    found = False
    if len(trees) > 0:
        trees = trees[np.argsort(np.sum(np.square(trees-[msxc-msxs,msyc-msys]),axis=1))]
        print('trying trees')
        for tree in trees:
            move_mouse(*(tree+[msxs,msys]))
            sleep(0.1)
            uptext,mask = uptext_mask(get_uptext())
            txt = image_to_string(mask)
            print('tesseract say:',txt)
            if 'Ima' in txt or 'unTan' in txt or 'm. 1.' in txt or '1.22' in txt or 'm. I.' in txt or 'm 1.' in txt or 'dnunTma' in txt or 'dnunTva' in txt:
                print('chop it down!')
                click_mouse(*(tree+[msxs,msys]))
                found = True
                inv = count_inv()
                print(inv)
                flag_wait()
                i = 0
                while inv == count_inv():
                    i += 1
                    if i > 10:
                        print('timed out')
                        break
                    sleep(0.5)
                else: #didn't timeout
                    break
    if found:        
        continue
    t = find_colors([55,79,25],minimap,mode='hsl',tol=0.05)
    t = filter_radius(t,[mmxc-mmxs,mmyc-mmys],65)
    if len(t) > 0:
        np.random.shuffle(t)
        click_mouse(*(t[0]+[mmxs,mmys]))
        flag_wait()